#### Importing libraries

In [1]:
import numpy as np
import p#### Combining Bothandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import openai

In [40]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [41]:
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/zhing/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/zhing/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from nltk.sentiment import SentimentIntensityAnalyzer
import re

#### feature engineering

In [27]:
def count_positive_negative(words, sia):
    pos_count = 0
    neg_count = 0
    
    # loop through words and check sentiment
    for word in words:
        sentiment_score = sia.polarity_scores(word)
        if sentiment_score['compound'] >= 0.5:
            pos_count += 1
        elif sentiment_score['compound'] <= -0.5:
            neg_count += 1
    return pos_count, neg_count

In [28]:
def count_positive_followed_by_negative(words, sia):
    # Initialize counters
    num_positive_followed_by_negative = 0

    # Iterate over words in input string
    for i, word in enumerate(words):
        # If current word is negative any of the previous 5 words are positive
        if sia.polarity_scores(word)['compound'] < 0 and any(sia.polarity_scores(w)['compound'] > 0 for w in words[max(0, i-5):i]):
            # Increment counter for positive words followed by negative words
            num_positive_followed_by_negative += 1

    return num_positive_followed_by_negative

In [6]:
def count_proper_nouns(pos_tags):
    num_proper_nouns = sum(1 for word, tag in pos_tags if tag == 'NNP')
    return num_proper_nouns

In [7]:
def count_conjunctions(pos_tags):
    num_conjunctions = sum(1 for word, tag in pos_tags if tag == 'CC')
    return num_conjunctions

In [59]:
def count_superlatives(pos_tags):
    num_superlatives = sum(1 for word, tag in pos_tags if (tag == "JJS" or tag == "RBS"))
    return num_superlatives

In [9]:
def count_pronouns(text):
    first_person = len(re.findall(r'\b(I|me|my|mine|we|us|our|ours)\b', text, flags=re.IGNORECASE))
    second_person = len(re.findall(r'\b(you|your|yours)\b', text, flags=re.IGNORECASE))
    third_person = len(re.findall(r'\b(he|him|his|she|her|hers|it|its|they|them|their|theirs)\b', text, flags=re.IGNORECASE))
    # first_person, second_person, third_person = count_pronouns(text)
    return first_person, second_person, third_person

In [10]:
def count_hedges(text):
    # Load the list of hedging words from a text file
    with open('hedging_words.txt', 'r') as f:
        hedging_words = [line.strip() for line in f]

    # Use NLTK to tokenize the text into words
    words = nltk.word_tokenize(text)

    # Count the number of hedging words in the text
    num_hedges = sum(1 for word in words if word.lower() in hedging_words)

    return num_hedges

In [11]:
def count_boosts(text):
    # Load the list of hedging words from a text file
    with open('boosting_words.txt', 'r') as f:
        boosting_words = [line.strip() for line in f]

    # Use NLTK to tokenize the text into words
    words = nltk.word_tokenize(text)

    # Count the number of hedging words in the text
    num_boosts = sum(1 for word in words if word.lower() in boosting_words)

    return num_boosts

In [12]:
def count_numbers(text):
    pattern = r"\d{1,3}(,\d{3})*(\.\d+)?"  # regular expression pattern to match numbers
    matches = re.findall(pattern, text)  # find all matches of the pattern in the text
    return len(matches)  # return the count of matches

In [35]:
def count_quotes(text):
    pattern = r"\s['].+['][.,!? ]?"
    matches = re.findall(pattern, text)
    return len(matches)

In [42]:
def count_words(words):
    return len(words)

In [43]:
def count_sentences(text):
    # Use the NLTK tokenizer to split the text into sentences
    sentences = nltk.sent_tokenize(text)
    return len(sentences)

In [50]:
def max_consecutive_upper(words):
    count = 0
    max_count = 0
    
    for word in words:
        if word.isupper():
            count += 1
        else:
            if count > max_count:
                max_count = count
            count = 0
            
    if count > max_count:
        max_count = count
    return max_count

In [53]:
def get_features(text):
    sia = SentimentIntensityAnalyzer()
    words = text.split()
    num_pos_words, num_neg_words = count_positive_negative(words, sia)
    num_positive_followed_by_negative = count_positive_followed_by_negative(words, sia)
    num_word = count_words(words)
    num_sentence = count_sentences(text)
    num_consecutive_upper = max_consecutive_upper(words)
    
    words = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    num_proper_nouns = count_proper_nouns(pos_tags)
    num_conjunctions = count_conjunctions(pos_tags)
    num_superlatives = count_superlatives(pos_tags)
    
    first_person, second_person, third_person = count_pronouns(text)
    num_hedges = count_hedges(text)
    num_boosts = count_boosts(text)
    num_numbers = count_numbers(text)
    num_quotes = count_quotes(text)
    
    return [num_word, num_sentence, num_pos_words, num_neg_words, num_positive_followed_by_negative, first_person, second_person, third_person, num_proper_nouns, num_conjunctions, num_superlatives, num_hedges, num_boosts, num_numbers, num_quotes, num_consecutive_upper]


#### Processing Data

In [25]:
training_data_filename = "../raw_data/fulltrain.csv"
df_train = pd.read_csv(traini#### feature engineeringng_data_filename, names=["label", "document"])

In [54]:
X_train = []
y_train = []

for index, row in df_train.iterrows():
    y_train.append(row["label"])
    text = row["document"]
    features = get_features(text)
    X_train.append(features)
    print(index, features)

0 [146, 7, 2, 0, 0, 0, 0, 2, 18, 7, 0, 9, 1, 0, 0, 1]
1 [122, 4, 0, 1, 1, 1, 0, 2, 10, 2, 1, 7, 2, 1, 1, 1]
2 [705, 28, 3, 2, 2, 2, 3, 8, 100, 23, 1, 21, 7, 24, 1, 1]
3 [705, 26, 5, 3, 3, 23, 3, 58, 37, 22, 2, 37, 8, 8, 1, 1]
4 [173, 7, 1, 1, 0, 6, 0, 9, 24, 5, 0, 10, 0, 1, 1, 1]
5 [93, 5, 0, 0, 0, 2, 0, 3, 14, 3, 0, 5, 3, 3, 1, 1]
6 [130, 6, 0, 0, 0, 0, 0, 8, 8, 3, 1, 3, 1, 1, 1, 0]
7 [687, 24, 1, 4, 1, 27, 1, 46, 30, 18, 0, 33, 4, 3, 0, 1]
8 [169, 4, 3, 1, 0, 6, 0, 7, 31, 6, 0, 5, 0, 8, 1, 1]
9 [84, 4, 0, 0, 0, 1, 0, 4, 13, 5, 0, 2, 0, 4, 1, 1]
10 [164, 8, 1, 0, 1, 1, 0, 9, 13, 2, 0, 7, 1, 8, 0, 1]
11 [707, 44, 1, 7, 3, 32, 2, 45, 61, 30, 2, 39, 3, 4, 1, 1]
12 [89, 4, 0, 0, 0, 1, 0, 1, 8, 3, 0, 1, 0, 3, 1, 0]
13 [36, 5, 1, 0, 0, 3, 0, 3, 2, 1, 0, 1, 0, 1, 0, 0]
14 [105, 4, 1, 1, 0, 2, 0, 2, 17, 1, 1, 1, 2, 0, 1, 1]
15 [142, 4, 4, 1, 1, 1, 0, 9, 11, 5, 0, 9, 0, 1, 0, 0]
16 [162, 7, 0, 0, 0, 8, 2, 5, 23, 6, 0, 16, 3, 1, 0, 1]
17 [186, 5, 4, 1, 1, 3, 0, 8, 13, 6, 0, 11, 1, 1, 0, 1]
18 [

153 [832, 39, 10, 8, 6, 33, 0, 17, 76, 21, 2, 22, 5, 10, 1, 2]
154 [747, 23, 6, 6, 1, 13, 0, 25, 70, 27, 7, 6, 3, 9, 0, 1]
155 [161, 5, 1, 0, 0, 11, 0, 9, 12, 3, 0, 5, 2, 1, 0, 1]
156 [192, 6, 1, 1, 2, 4, 0, 6, 17, 10, 0, 8, 2, 4, 1, 2]
157 [144, 6, 1, 2, 0, 2, 0, 7, 18, 8, 0, 3, 1, 0, 1, 1]
158 [115, 4, 0, 0, 0, 3, 0, 4, 15, 3, 1, 3, 0, 4, 1, 0]
159 [167, 7, 2, 0, 3, 11, 0, 6, 7, 1, 0, 9, 2, 3, 1, 1]
160 [89, 4, 0, 3, 1, 2, 0, 3, 14, 1, 0, 4, 1, 4, 1, 1]
161 [704, 23, 0, 7, 1, 6, 1, 39, 29, 13, 0, 28, 5, 11, 1, 1]
162 [407, 23, 1, 0, 1, 15, 1, 20, 59, 14, 0, 9, 6, 5, 1, 1]
163 [133, 4, 0, 0, 0, 0, 0, 8, 7, 3, 0, 2, 1, 1, 0, 1]
164 [110, 5, 0, 1, 2, 0, 0, 4, 12, 3, 0, 1, 1, 6, 1, 0]
165 [160, 5, 3, 0, 3, 1, 0, 6, 24, 6, 1, 3, 0, 7, 1, 1]
166 [503, 25, 5, 7, 1, 15, 0, 9, 58, 13, 1, 17, 2, 15, 1, 1]
167 [142, 6, 6, 0, 1, 2, 1, 13, 15, 7, 1, 0, 0, 8, 1, 1]
168 [607, 20, 2, 4, 4, 10, 5, 21, 32, 21, 0, 14, 3, 7, 0, 1]
169 [166, 5, 0, 2, 0, 4, 0, 5, 18, 5, 2, 7, 1, 4, 0, 1]
170 [627, 38, 1, 

304 [715, 37, 2, 3, 6, 28, 0, 14, 97, 26, 0, 23, 1, 6, 1, 3]
305 [78, 5, 0, 0, 0, 1, 0, 5, 22, 1, 1, 0, 0, 0, 1, 0]
306 [590, 20, 7, 0, 2, 9, 2, 26, 52, 20, 2, 29, 3, 7, 1, 1]
307 [758, 57, 8, 2, 2, 40, 2, 33, 57, 27, 1, 39, 4, 10, 1, 1]
308 [568, 26, 8, 1, 2, 6, 1, 23, 7, 15, 3, 32, 3, 0, 1, 1]
309 [874, 50, 10, 8, 6, 16, 1, 55, 52, 36, 0, 27, 9, 8, 1, 1]
310 [631, 42, 4, 6, 0, 19, 3, 28, 64, 24, 2, 21, 5, 1, 1, 1]
311 [92, 5, 1, 0, 0, 3, 0, 4, 12, 2, 0, 3, 1, 2, 1, 1]
312 [191, 5, 3, 2, 4, 1, 0, 15, 6, 4, 0, 10, 2, 1, 0, 0]
313 [75, 5, 0, 0, 0, 2, 0, 3, 9, 3, 0, 5, 1, 0, 1, 0]
314 [582, 28, 2, 5, 6, 27, 4, 19, 74, 17, 1, 20, 1, 6, 1, 3]
315 [651, 45, 3, 2, 0, 12, 11, 18, 73, 21, 0, 25, 6, 8, 1, 1]
316 [149, 5, 2, 0, 2, 1, 0, 5, 18, 5, 0, 1, 0, 3, 1, 1]
317 [111, 6, 4, 1, 1, 0, 0, 3, 11, 1, 0, 1, 1, 3, 1, 0]
318 [209, 5, 1, 3, 2, 2, 3, 8, 6, 10, 0, 10, 1, 0, 0, 0]
319 [850, 45, 9, 23, 6, 24, 2, 20, 86, 23, 2, 25, 2, 24, 1, 2]
320 [158, 7, 2, 1, 0, 1, 1, 5, 19, 2, 0, 4, 2, 2, 0, 1]
321

455 [580, 22, 4, 6, 6, 6, 1, 37, 38, 18, 0, 23, 2, 2, 1, 1]
456 [161, 4, 3, 0, 0, 4, 0, 10, 19, 5, 1, 12, 1, 0, 1, 1]
457 [102, 6, 1, 1, 1, 1, 2, 3, 17, 1, 1, 2, 0, 2, 1, 1]
458 [171, 4, 1, 1, 0, 7, 0, 10, 13, 6, 0, 8, 0, 6, 1, 1]
459 [112, 5, 1, 1, 1, 3, 0, 4, 11, 3, 0, 3, 1, 1, 0, 1]
460 [159, 6, 2, 0, 0, 3, 1, 11, 14, 5, 0, 10, 4, 5, 1, 3]
461 [620, 25, 3, 15, 5, 14, 17, 21, 34, 16, 0, 17, 3, 8, 1, 4]
462 [755, 35, 5, 16, 3, 11, 10, 11, 75, 29, 2, 20, 6, 5, 1, 1]
463 [233, 4, 4, 0, 3, 1, 0, 17, 9, 4, 0, 5, 2, 1, 0, 1]
464 [77, 4, 3, 0, 1, 2, 0, 7, 17, 0, 1, 3, 0, 0, 1, 1]
465 [158, 5, 4, 3, 1, 4, 0, 7, 21, 7, 0, 2, 1, 2, 1, 1]
466 [132, 4, 4, 0, 1, 0, 0, 4, 17, 3, 3, 2, 0, 5, 0, 1]
467 [181, 6, 5, 0, 0, 10, 0, 14, 6, 7, 2, 7, 0, 2, 0, 0]
468 [125, 5, 0, 0, 0, 2, 0, 1, 8, 5, 0, 0, 0, 2, 1, 1]
469 [58, 4, 0, 0, 0, 0, 2, 5, 7, 3, 0, 3, 1, 2, 0, 1]
470 [222, 4, 2, 0, 0, 1, 0, 3, 16, 8, 0, 4, 1, 2, 0, 0]
471 [600, 42, 1, 0, 0, 28, 6, 20, 57, 17, 0, 22, 0, 3, 1, 1]
472 [502, 16, 4, 1, 1, 

606 [612, 28, 2, 6, 7, 29, 4, 24, 46, 17, 2, 22, 3, 7, 1, 1]
607 [179, 7, 2, 0, 0, 8, 1, 8, 18, 4, 0, 6, 1, 0, 1, 1]
608 [411, 15, 11, 1, 3, 10, 0, 22, 49, 13, 5, 11, 3, 3, 0, 1]
609 [131, 3, 1, 0, 0, 2, 2, 7, 12, 4, 0, 2, 7, 1, 1, 0]
610 [192, 8, 2, 0, 1, 9, 3, 12, 15, 4, 0, 12, 4, 0, 1, 1]
611 [114, 5, 3, 1, 1, 0, 6, 8, 14, 3, 0, 3, 1, 0, 1, 1]
612 [170, 6, 2, 1, 1, 3, 0, 12, 12, 9, 0, 5, 2, 2, 0, 1]
613 [724, 41, 10, 4, 4, 26, 1, 15, 88, 26, 1, 22, 5, 15, 1, 1]
614 [89, 5, 0, 0, 0, 3, 1, 5, 19, 2, 0, 3, 2, 2, 1, 1]
615 [145, 7, 1, 0, 0, 6, 0, 3, 26, 3, 0, 5, 1, 5, 1, 1]
616 [715, 39, 0, 3, 0, 14, 1, 26, 60, 29, 1, 17, 5, 5, 1, 1]
617 [113, 5, 2, 7, 1, 2, 0, 3, 23, 1, 0, 2, 2, 5, 1, 1]
618 [523, 21, 2, 9, 5, 2, 0, 32, 23, 19, 2, 20, 1, 8, 0, 1]
619 [679, 30, 4, 1, 1, 4, 2, 20, 45, 19, 3, 19, 3, 5, 1, 1]
620 [360, 17, 3, 3, 0, 6, 0, 7, 41, 6, 1, 5, 0, 12, 1, 1]
621 [424, 16, 7, 0, 3, 16, 7, 23, 19, 15, 2, 28, 7, 1, 1, 1]
622 [156, 5, 3, 0, 3, 4, 3, 3, 26, 7, 1, 5, 1, 0, 1, 1]
623 [567

...

48196 [237, 7, 2, 0, 0, 0, 0, 2, 16, 11, 0, 5, 1, 16, 1, 0]
48197 [237, 7, 2, 0, 0, 0, 0, 2, 16, 11, 0, 5, 1, 16, 1, 0]
48198 [128, 7, 0, 1, 0, 0, 0, 1, 16, 2, 1, 2, 0, 13, 0, 1]
48199 [156, 10, 0, 1, 0, 1, 0, 3, 11, 10, 0, 1, 1, 17, 0, 1]
48200 [198, 6, 1, 0, 0, 1, 0, 1, 21, 6, 0, 5, 1, 14, 0, 1]
48201 [256, 10, 2, 2, 1, 4, 0, 3, 30, 0, 1, 9, 1, 5, 1, 1]
48202 [344, 11, 1, 2, 0, 2, 0, 3, 28, 8, 2, 6, 0, 12, 1, 1]
48203 [346, 11, 2, 2, 3, 0, 0, 2, 57, 11, 0, 9, 0, 22, 0, 1]
48204 [203, 9, 2, 0, 1, 0, 0, 4, 40, 1, 0, 5, 1, 4, 1, 0]
48205 [377, 13, 4, 3, 1, 0, 0, 4, 92, 10, 0, 5, 1, 9, 0, 1]
48206 [359, 14, 2, 3, 0, 0, 0, 17, 45, 11, 1, 6, 1, 8, 1, 1]
48207 [311, 15, 1, 2, 2, 0, 0, 3, 27, 7, 0, 14, 0, 19, 0, 1]
48208 [430, 13, 0, 1, 1, 1, 0, 7, 59, 13, 0, 24, 1, 2, 1, 2]
48209 [250, 8, 3, 0, 2, 0, 0, 6, 20, 6, 1, 16, 2, 9, 1, 1]
48210 [305, 8, 2, 2, 5, 1, 0, 8, 24, 7, 3, 3, 0, 17, 1, 1]
48211 [549, 17, 3, 0, 2, 1, 0, 11, 105, 11, 0, 10, 1, 21, 1, 1]
48212 [378, 14, 3, 3, 1, 2, 0, 13, 41,

48348 [121, 3, 1, 2, 2, 0, 0, 1, 19, 4, 0, 2, 0, 3, 0, 1]
48349 [38, 1, 0, 0, 0, 0, 0, 0, 23, 0, 0, 0, 0, 0, 0, 1]
48350 [278, 12, 2, 0, 1, 0, 0, 5, 22, 8, 0, 2, 3, 8, 1, 1]
48351 [124, 5, 0, 0, 0, 0, 0, 2, 19, 3, 1, 1, 0, 7, 0, 2]
48352 [188, 8, 0, 0, 0, 0, 0, 5, 51, 2, 0, 5, 0, 11, 1, 1]
48353 [270, 10, 7, 3, 1, 1, 0, 7, 30, 6, 0, 6, 0, 3, 1, 1]
48354 [102, 4, 0, 0, 0, 0, 0, 4, 17, 3, 0, 4, 0, 5, 0, 1]
48355 [165, 6, 1, 7, 2, 0, 0, 1, 23, 6, 0, 5, 1, 4, 1, 1]
48356 [175, 5, 1, 1, 1, 2, 0, 6, 45, 3, 0, 5, 1, 0, 1, 0]
48357 [183, 5, 0, 3, 0, 3, 0, 1, 20, 7, 0, 7, 2, 2, 1, 1]
48358 [32, 1, 0, 1, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0]
48359 [214, 6, 0, 0, 0, 1, 0, 2, 49, 4, 0, 3, 0, 3, 1, 1]
48360 [101, 5, 1, 1, 0, 0, 0, 2, 19, 0, 0, 1, 0, 5, 1, 1]
48361 [85, 5, 0, 0, 1, 0, 0, 1, 8, 3, 0, 0, 0, 10, 0, 1]
48362 [511, 11, 1, 7, 1, 1, 0, 6, 50, 18, 2, 9, 5, 1, 1, 1]
48363 [200, 8, 1, 0, 1, 0, 0, 5, 31, 3, 5, 0, 0, 15, 0, 0]
48364 [332, 16, 2, 0, 0, 1, 0, 9, 30, 14, 0, 9, 1, 12, 1, 0]
48365 [14

48492 [569, 22, 2, 5, 3, 0, 0, 12, 102, 10, 2, 5, 0, 18, 1, 2]
48493 [389, 17, 0, 1, 1, 1, 0, 3, 45, 13, 1, 5, 2, 32, 1, 1]
48494 [196, 7, 1, 0, 0, 0, 0, 7, 2, 2, 0, 7, 0, 12, 1, 1]
48495 [229, 6, 2, 0, 0, 0, 0, 1, 48, 8, 0, 7, 0, 7, 0, 1]
48496 [345, 13, 6, 2, 1, 0, 0, 3, 64, 5, 0, 5, 0, 26, 0, 1]
48497 [258, 12, 2, 0, 2, 1, 0, 6, 24, 6, 1, 6, 0, 3, 1, 0]
48498 [195, 8, 2, 0, 0, 0, 0, 7, 36, 3, 0, 2, 0, 3, 0, 1]
48499 [288, 9, 1, 1, 0, 0, 0, 4, 57, 8, 0, 9, 1, 3, 1, 1]
48500 [638, 20, 1, 1, 3, 1, 0, 21, 62, 14, 0, 12, 1, 31, 1, 1]
48501 [225, 7, 1, 0, 1, 0, 0, 4, 16, 9, 0, 4, 0, 3, 1, 0]
48502 [233, 10, 2, 0, 0, 0, 0, 5, 33, 5, 0, 3, 1, 15, 1, 0]
48503 [279, 11, 0, 1, 0, 1, 0, 7, 43, 4, 0, 9, 0, 17, 0, 2]
48504 [287, 11, 1, 0, 0, 0, 0, 7, 40, 6, 1, 3, 0, 10, 0, 1]
48505 [631, 23, 0, 3, 0, 7, 3, 19, 43, 24, 0, 20, 2, 10, 1, 1]
48506 [75, 4, 1, 0, 0, 1, 0, 0, 10, 0, 0, 2, 0, 13, 1, 1]
48507 [191, 8, 0, 1, 0, 0, 0, 2, 39, 8, 0, 1, 0, 15, 0, 1]
48508 [182, 8, 3, 0, 0, 0, 0, 2, 28, 5, 0, 2

48632 [293, 12, 0, 1, 0, 1, 0, 3, 28, 3, 1, 4, 0, 21, 0, 1]
48633 [173, 8, 2, 1, 2, 0, 0, 3, 14, 9, 1, 7, 1, 2, 1, 1]
48634 [236, 8, 1, 1, 1, 0, 0, 11, 37, 6, 0, 2, 0, 1, 1, 1]
48635 [255, 10, 2, 0, 1, 0, 0, 8, 32, 6, 0, 7, 1, 11, 0, 1]
48636 [208, 7, 0, 0, 1, 1, 0, 8, 33, 11, 0, 5, 2, 3, 1, 1]
48637 [608, 25, 1, 0, 0, 7, 0, 19, 73, 20, 2, 19, 0, 7, 1, 1]
48638 [179, 9, 4, 0, 1, 1, 0, 3, 8, 4, 0, 4, 0, 6, 1, 0]
48639 [383, 10, 3, 2, 1, 1, 0, 6, 79, 13, 0, 8, 1, 8, 1, 1]
48640 [514, 20, 6, 0, 1, 0, 0, 14, 91, 20, 0, 19, 1, 10, 1, 1]
48641 [49, 3, 0, 0, 0, 1, 0, 0, 13, 1, 0, 0, 0, 5, 0, 2]
48642 [286, 12, 0, 4, 1, 5, 0, 4, 44, 6, 0, 4, 0, 15, 0, 1]
48643 [42, 2, 0, 0, 0, 1, 0, 1, 7, 1, 0, 0, 0, 7, 0, 1]
48644 [211, 7, 0, 0, 0, 8, 0, 2, 36, 4, 1, 6, 0, 15, 0, 2]
48645 [556, 19, 4, 3, 7, 6, 0, 12, 122, 7, 1, 16, 4, 5, 1, 1]
48646 [280, 12, 3, 0, 4, 0, 0, 8, 31, 8, 0, 8, 0, 6, 0, 1]
48647 [343, 13, 0, 2, 0, 0, 0, 8, 24, 9, 2, 10, 3, 21, 0, 1]
48648 [306, 11, 0, 1, 1, 0, 0, 2, 45, 12, 1, 10,

48778 [353, 11, 2, 0, 0, 7, 0, 2, 42, 8, 0, 8, 0, 14, 0, 1]
48779 [277, 12, 1, 0, 0, 0, 0, 15, 9, 8, 1, 6, 0, 23, 0, 0]
48780 [282, 9, 2, 2, 1, 0, 0, 7, 41, 10, 0, 6, 3, 3, 1, 1]
48781 [144, 4, 0, 1, 0, 2, 0, 3, 15, 2, 1, 0, 0, 19, 0, 1]
48782 [204, 8, 0, 0, 0, 0, 0, 6, 27, 7, 0, 5, 0, 4, 1, 1]
48783 [337, 13, 0, 0, 2, 1, 0, 11, 48, 10, 0, 8, 0, 10, 0, 2]
48784 [234, 9, 3, 1, 1, 0, 0, 6, 33, 5, 0, 5, 0, 11, 0, 1]
48785 [234, 9, 3, 1, 1, 0, 0, 6, 33, 5, 0, 5, 0, 11, 0, 1]
48786 [284, 8, 0, 0, 0, 7, 0, 5, 39, 8, 0, 3, 0, 29, 0, 1]
48787 [229, 10, 0, 2, 1, 0, 0, 10, 28, 8, 2, 7, 4, 2, 0, 1]
48788 [74, 1, 0, 0, 0, 3, 0, 0, 15, 0, 0, 0, 0, 29, 0, 5]
48789 [180, 5, 0, 1, 0, 0, 0, 2, 18, 9, 0, 8, 3, 3, 0, 1]
48790 [156, 5, 0, 0, 0, 0, 0, 1, 30, 4, 0, 5, 0, 5, 0, 1]
48791 [169, 5, 0, 0, 0, 1, 0, 1, 39, 5, 0, 5, 0, 13, 0, 1]
48792 [252, 14, 2, 1, 2, 2, 0, 5, 13, 5, 0, 10, 0, 12, 1, 2]
48793 [91, 2, 0, 0, 0, 3, 0, 0, 21, 0, 0, 0, 0, 29, 0, 5]
48794 [486, 16, 4, 0, 0, 0, 0, 11, 68, 16, 1, 13, 2, 

#### Exporting features engineered

In [57]:
import csv

def write_csv(data, headers, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)
        for row in data:
            writer.writerow(row)

In [58]:
headers = ['num_word', 'num_sentence', 'num_pos_words', 'num_neg_words', 'num_pos_followed_by_neg', 'first_person', 'second_person', 'third_person', 'num_proper_nouns', 'num_conjunctions', 'num_superlatives', 'num_hedges', 'num_boosts', 'num_numbers', 'num_quotes', "num_consecutive_upper"]
filename = 'contextualFeatures1.csv'

write_csv(X_train, headers, filename)